# Múltiples mediciones

Al realizar múltiples mediciones,
podemos mejorar la precisión de la magnitud que estamos midiendo.
Hay varias formas de realizar múltiples mediciones,
de las cuales consideraremos 3 casos.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

Imaginen que tenemos $N=10$ objetos idénticos,
que vamos a representar con un rectángulo,
y queremos medir su longitud.

In [ ]:
WIDTH = 1


def add_rectangles(*, width=WIDTH, n_rectangles=10, ymin=-0.5):
    ymax = 1.5
    fig, ax = plt.subplots(figsize=(12, (ymax - ymin) / 2), dpi=100)
    for x in range(n_rectangles):
        rect = plt.Rectangle(
            (width * x, 0),
            width,
            1,
            facecolor="none",
            edgecolor="black",
        )
        ax.add_patch(rect)
    ax.set(
        xlim=(-0.5, width * n_rectangles + 0.5),
        ylim=(ymin, 1.5),
        xticks=(),
        yticks=(),
    )
    ax.spines.clear()
    return ax


ax = add_rectangles()

Vamos a asumir que son exactamente iguales,
es decir,
si tuviésemos infinita precisión para medir,
mediríamos lo mismo para todos.

Estos objetos no representan necesariamente rectángulos físicos,
sino que pueden representar intervalos de tiempo.

## Caso 1: mediciones independientes

Una primera opción es medirlos de forma independiente:

In [ ]:
ax = add_rectangles(ymin=-3)
for i in range(10):
    ax.errorbar(
        x=WIDTH * (i + 0.5),
        y=-(i % 2) - 1,
        xerr=WIDTH / 2,
        capsize=5,
    )

En ese caso,
tenemos dos posibilidades,
según si estamos limitados por la resolución instrumental o no.

### Con error por resolución instrumental

Si el error está dado por la resolución $\Delta L$ de nuestro instrumento,
vamos a medir siempre lo mismo.
Hacer más mediciones no aporta nada.
En este caso,
el error será:

$$ \Delta L $$

### Con error estadístico

::::{margin}
:::{note}
El error estadístico tiene que tener promedio $0$.
Coloquialmente,
tener las mismas chances de medir por exceso que por defecto.
:::
::::

En cambio,
si en cada medición tenemos un error estadístico $\sigma_L$,
podemos promediar las mediciones y disminuir el error.
El error del promedio $\langle L \rangle$ disminuye como

$$ \sigma_{\langle L \rangle} = \frac{\sigma_L}{\sqrt N} $$

::::{margin}
:::{note}
Técnicamente,
$\sigma_L$ depende del error instrumental $\Delta L$

$$\sigma_L^2 = \sigma_{L_0}^2 + (\Delta L)^2 / 12$$

donde $\sigma_{L_0}$ es la desviación estándar de las mediciones no limitadas por la resolución.
Es decir,
con todos los decimales.
Sin embargo,
casi siempre estimamos $\sigma_L$ directamente a partir de mediciones
ya "redondeadas" por el instrumento.
:::
::::

El error $\sigma_{\langle L \rangle}$ de este promedio $\langle L \rangle$
no depende de la resolución instrumental $\Delta L$,
si $\sigma_L \gtrsim \Delta L$.
Pero no significa que podamos hacerlo tan chico como queramos,
ya que sí depende de la calibración instrumental.
Por ejemplo,
un cronómetro típico tiene una resolución instrumental de $10 \text{ ms} = 0.01 \text{ s} = 10^{-2} \text{ s}$,
pero tiene un error de calibración del orden de $\sim 5 \text{ μs} = 5 \times 10^{-6} \text{ s}$.
Midiendo intervalos de tiempo a mano,
donde se tiene un error del orden de $\sigma_T \approx 50 \text{ ms}$,
hay que tener cuidado de no promediar más de $N = \left(\frac{50 \text{ ms}}{5 \text{ μs}}\right)^2 = 10^8$ veces.

:::{warning}
Probablemente se rompan otras hipótesis de la teoría antes de llegar a ese nivel de precisión.
Por ejemplo,
en el caso del péndulo,
la aproximación de pequeñas oscilaciones.
:::

## Caso 2: midiendo extremos

Si alineamos los $N$ objetos,
podemos calcular indirectamente el largo $L_1$ de un objeto
a partir del largo $L_N$ de $N$ objetos consecutivos.

$$ L_1 = \frac{L_N}{N} $$

In [ ]:
ax = add_rectangles(ymin=-2)
ax.errorbar(
    x=5 * WIDTH,
    y=-1,
    xerr=10 * WIDTH / 2,
    capsize=10,
)

Supongamos que el error de medir **directamente** $L_1$,
es igual al de medir directamente $L_N$:

$$ \sigma_{L_1} = \sigma_{L_N} = \sigma_L $$

Entonces,
por propagación de errores,
medir **indirectamente** $L_1$ es:

$$ \sigma_{L_1} = \frac{\sigma_{L}}{N} $$

que disminuye más rápido que el promedio.

Repitiendo la cuenta anterior,
al medir a mano con un cronómetro,
si $N = \left(\frac{50 \text{ ms}}{5 \text{ μs}}\right)^1 = 10^4 = 10.000$,
tendríamos que tener en cuenta el error de calibración.

### Ejemplo del caso 2

Imaginemos que la sección mide $L_1^{real} = 10.12 \text{ mm}$,
pero medimos con una regla de resolución $1 \text{ mm}$.
Si medimos una única sección,
nuestra medición va ser $L_1^{medido} = (10 \pm 1) \text{ mm}$.
Pero,
si medimos $N=10$ secciones consecutivas,
cuyo largo es

$$ L_{10}^{real}
= 10 \times 10.12 \text{ mm}
= 101.2 \text{ mm}
$$

vamos a medir $L_{10}^{medido} = 101 \pm 1 \text{ mm}$.
Noten que es diferente a multiplicar por $N=10$
la medición de una sección:

$$ 10 \times L_1^{medido}
= 10 \times (10 \pm 1) \text{ mm}
= (100 \pm 10) \text{ mm}
$$
Calculando indirectamente $L_1$,
y propagando,
llegamos a:

$$ L_1^{indirecto} = 10.1 \pm 0.1 \text{ mm} $$

que tiene un error 10 veces menor
que la medición directa de $L_1$.
¡Ganamos un decimal!

## Caso 3: mediciones consecutivas

En el caso 3,
al igual que el caso 2,
alineamos todos los objetos para medirlos.
Pero,
medimos todas las distancias intermedios $L(n)$:

In [ ]:
ax = add_rectangles(ymin=-2.5)
for i in range(1, 11):
    ax.errorbar(x=0, y=-1, xerr=[[0], [i * WIDTH]], capsize=10, color="C0")

¿Cómo calculamos el largo $L_1$ de un segmento a partir de estas mediciones?

### Opción 1 (incorrecta)

::::{margin}
:::{note}
Ojo, este $\Delta L_n$ no es un error de medición.
:::
::::

Podemos tomar las diferencias entre puntos consecutivos:

$$ \Delta L_n = L(n) - L(n-1) $$

y promediarlas.

¿Si hacemos un promedio,
el error no debería disminuir como $1 / \sqrt{N}$?
En ese caso, ¿no es mejor el caso anterior,
donde solo mediamos los extremos
y el error disminuía como $1 / N$?

Como los $\Delta L_n$ no son independientes,
no se cumplen las hipótesis
para que el error del promedio sea $\sigma_L / \sqrt{N}$.

Si escribimos explícitamente los $\Delta L_n$:

$$ \begin{cases}
\Delta L_1 = L(1) - L(0) \\
\Delta L_2 = L(2) - L(1) \\
\ldots
\end{cases} $$

vemos que $\Delta L_1$ y $\Delta L_1$ comparten la medición $L(1)$.
Si hacemos un promedio de estos:

$$ \frac{\Delta L_2 + \Delta L_1}{2}
= \frac{[L(2) - L(1)] + [L(1) - L(0)]}{2}
= \frac{L(2) - L(0)}{2}
$$

quedan solo los extremos $L(2)$ y $L(0)$.

Si lo extendemos a las $N$ mediciones,

$$ \frac{1}{N} \sum_{n=1}^N \Delta L_n = \frac{L(n) - L(0)}{N} $$

llegamos a lo mismo que calculábamos en el caso anterior.

### Opción 2

Para combinar las mediciones $L(n)$,
aprovechando la información de las mediciones intermedias,
y obtener $L_1$,
se puede realizar un ajuste lineal:

$$ L(n) = L_1 n + L_o $$

donde $n$ es la cantidad de secciones consecutivas.

In [ ]:
x = np.arange(1, 10)
plt.figure(figsize=(6, 3))
plt.plot(x, WIDTH * x, "o-")
plt.xlabel("Cantidad de segmentos $n$")
plt.ylabel("Largo $L_n$")
None

En este caso,
el error de $L_1$ disminuye aún más rápido que en el caso anterior.
Según {footcite:t}`bevington2003data`,
ecuaciones (6.23) y (6.13):

$$ \sigma_{L_1}^2 = \frac{N \sigma_L^2}{N \sum_n n^2 + (\sum_n n)^2} $$

Dado que $n = \{1, \ldots, N\}$,

$$
\begin{cases}
\sum_{n=1}^N n = \frac{N^2 + N}{2} \\ \\
\sum_{n=1}^N n^2 = \frac{2N^3 + 3N + N}{6}
\end{cases}
$$

Reemplazando en la ecuación anterior:

$$ \sigma_{L_1}
\sim \frac{\sigma_L}{N^{3/2}}
$$

Repitiendo la cuenta para el cronómetro,
si $N \approx 500$,
tendríamos que tener en cuenta el error de calibración.
Ya no es (tan) inalcanzable.

### Aclaración importante

Hay un detalle importante,
que lo diferencia de simplemente repetir $N$ veces el caso 2,
una vez para cada $L(n)$:

In [ ]:
ax = add_rectangles(ymin=-5.5)
for i in range(1, 11):
    ax.errorbar(x=0, y=-i / 2, xerr=[[0], [i * WIDTH]], capsize=5, color="C1")

En el caso 3,
las mediciones comparten el mismo origen,
mientras que en el esquema anterior,
estamos volviendo a medir el origen.

Es más claro con un ejemplo.
Para mediciones con una regla,
se puede:
1. apoyar la regla una vez y tomar todas las mediciones intermedias
2. apoyar la regla $N$ veces, y tomar cada vez un $L(n)$ distinto.

En este último caso,
cada vez que apoyamos la regla,
estamos introduciendo un error en el origen.

## Bibliografía

```{footbibliography}
```